# Deterministic Uncertainty Sets - Multiple uncertain constraints, polyhedral uncertainty

In [1]:
import cvxpy as cp
import scipy as sc
from sklearn import datasets
import numpy as np
import numpy.random as npr
import numpy.testing as npt
import torch
import pandas as pd
import lropt
import matplotlib.pyplot as plt
RTOL = 1e-04
ATOL = 1e-04
import warnings
warnings.filterwarnings("ignore")
plt.rcParams.update({
    "text.usetex":True,
    "font.size":18,
    "font.family": "serif"
})
colors = ["tab:blue", "tab:green", "tab:orange", 
          "tab:red", "tab:purple", "tab:brown", "tab:pink", "tab:grey", "tab:olive"]

We solve the problem
$$
\begin{array}{ll}
\text{minimize} & c^Tx\\
\text{subject to}  & (P_1u + a)^Tx  \leq  10\\
& (P_2u)^Tx  \leq  5\\
& x \leq 5,
\end{array}
$$
where $c$, $P_1$, $P_2$, and $a$ are constants, and $u$ is the uncertain parameter from a polyhedral uncertainty set

In [2]:
m = 5
#restate the ellipsoidal set 
D =  np.vstack((np.eye(m), -2*np.eye(m)))
d = np.concatenate((0.1*np.ones(m), 0.1*np.ones(m)))
poly_u = lropt.UncertainParameter(m, 
                                  uncertainty_set = lropt.Polyhedral(lhs = D, rhs = d))
n = 4
# formulate cvxpy variable
x_r = cp.Variable(n)

# formulate problem constants
P1 = 0.5 * np.eye(m)[:n, :]
P2 = 3*np.random.rand(n,m)
a = 0.1 * np.random.rand(n)
c = np.random.rand(n)

# formulate objective
objective = cp.Minimize(-c@x_r)

# formulate constraints
constraints = [(P1@poly_u +a)@ x_r <=10, x_r<=5]
constraints += [(P2@poly_u)@ x_r <= 5]

# formulate Robust Problem
prob_robust = lropt.RobustProblem(objective, constraints)

# solve

# solve
prob_robust.solve()
print("LRO objective value: ", prob_robust.objective.value, "\nLRO x: ", x_r.value)

LRO objective value:  -6.344703977452691 
LRO x:  [  5.           5.           4.52484486 -10.516037  ]


We compare it with its direct reformulation, where each uncertain constraint is treated independently,
$$
\begin{array}{ll}
\text{minimize} & c^Tx\\
\text{subject to}  & a^Tx + w_1^Td  \leq  10\\
& w_1^TD = P_1^Tx\\
& w_2^Td  \leq  5\\
& w_2^TD = P_2^Tx\\
& w_1, w_2 \geq 0\\
& x \leq 5,
\end{array}
$$
where $w_1$, $w_2$ are new variables introduced.

In [3]:
# formulate using cvxpy
x_cvxpy = cp.Variable(n)
w1 = cp.Variable(2*m)
w2 = cp.Variable(2*m)
# formulate objective
objective = cp.Minimize(-c@x_cvxpy)

# formulate constraints
constraints = [a@x_cvxpy + w1@d <= 10]
constraints += [w1@D == P1.T@x_cvxpy]
constraints += [w2@d <= 5]
constraints += [w2@D == P2.T@x_cvxpy]
constraints += [w1>=0, w2 >= 0, x_cvxpy <=5]
# formulate Robust Problem
prob_cvxpy = cp.Problem(objective, constraints)

# solve
prob_cvxpy.solve()
print("Cvxpy objective value: ", prob_cvxpy.objective.value, "\nCvxpy x: ", x_cvxpy.value)

# assert x values are equal
npt.assert_allclose(x_r.value, x_cvxpy.value, rtol=RTOL, atol=ATOL)

Cvxpy objective value:  -6.344703981544437 
Cvxpy x:  [  5.           5.           4.52484485 -10.51603696]
